In [1]:
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '8g'), ('spark.driver.memory','15g'),("spark.driver.maxResultSize","0"),("spark.executor.cores", '2'),("spark.executor.instances", "12"),("spark.dynamicAllocation.enabled","False")])

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4,application_1649396207307_0007,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
import pyspark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
from pyspark.storagelevel import StorageLevel

from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml import Transformer, classification
from pyspark.ml import tuning
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df = sqlContext.read.parquet("hdfs:///NLP-parquet-processed/").repartition(4800)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df = df.withColumnRenamed("stars","label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#df = df.localCheckpoint()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- label: double (nullable = true)
 |-- text: string (nullable = true)
 |-- stemmed: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Bigrams: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Trigrams: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- featuresUni: vector (nullable = true)
 |-- featuresBi: vector (nullable = true)
 |-- featuresTri: vector (nullable = true)

In [15]:
dfUni = df.select("label","featuresUni").withColumnRenamed("featuresUni","features").repartition(3200).cache()
dfBi = df.select("label","featuresBi").withColumnRenamed("featuresBi","features").repartition(3200).cache()
dfTri = df.select("label","featuresTri").withColumnRenamed("featuresTri","features").repartition(3200).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
lr= classification.LogisticRegression(maxIter=75, regParam=0.01, elasticNetParam=0, tol=1E-6, standardization=False, family="binomial")
lrBi = classification.LogisticRegression(maxIter=75, regParam=0.01, elasticNetParam=0, tol=1E-6, standardization=False, family="binomial")
lrTri = classification.LogisticRegression(maxIter=75, regParam=0.01, elasticNetParam=0, tol=1E-6, standardization=False, family="binomial")
ovrUni = classification.OneVsRest(classifier=lr)
ovrBi = classification.OneVsRest(classifier=lrBi)
ovrTri = classification.OneVsRest(classifier=lrTri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Declare evaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")

### One Vs Rest Unigram

In [ ]:
ovrModelUni = ovrUni.fit(dfUni)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
resultUni = ovrModelUni.transform(dfUni)

In [ ]:
resultUni.printSchema()

#### Metric

In [ ]:
evaluator.evaluate(resultUni,{evaluator.metricName: "accuracy"})

In [ ]:
ovrModelUni.save('s3://dataset-26-proj/yelp_academic_dataset_review_ovrUni/ovrUniV1')

### One Vs Rest Bigram

In [10]:
ovrModelBi = ovrBi.fit(dfBi)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
resultBi = ovrModelBi.transform(dfBi)

In [ ]:
resultBi.printSchema()

#### Metric

In [ ]:
evaluator.evaluate(resultBi,{evaluator.metricName: "accuracy"})

In [ ]:
ovrModelBi.save('s3://dataset-26-proj/yelp_academic_dataset_review_ovrBi/ovrBiV1')

### One Vs Rest Trigram

In [17]:
ovrModelTri = ovrTri.fit(dfTri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
resultTri = ovrModelTri.transform(dfTri)

In [ ]:
resultTri.printSchema()

#### Metric

In [ ]:
evaluator.evaluate(resultTri,{evaluator.metricName: "accuracy"})

In [ ]:
ovrModelTri.save('s3://dataset-26-proj/yelp_academic_dataset_review_ovrTri/ovrTriV1')

### Test

In [ ]:

# paramGrid = tuning.ParamGridBuilder().addGrid(lr.maxIter, [30,50,75]) \
#                               .addGrid(lr.regParam, [0.03, 0.04, 0.05]) \
#                               .addGrid(lr.elasticNetParam, [0.1,0.2,0.3]) \
#                               .build()
# evaluator = MulticlassClassificationEvaluator()
# valid = tuning.TrainValidationSplit(estimator=lr,
#                              estimatorParamMaps=paramGrid,
#                              evaluator=evaluator,
#                              trainRatio=0.80)
# model = valid.fit(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-12:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1397



In [14]:
best_model = model.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#rf_model = rf.fit(df)

#lrModel = ovr.fit(df)

In [13]:
#best_model.printSchema()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- prediction: double (nullable = true)

In [7]:
sc.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…